In [322]:
import numpy as np 
import random
import pprint
from itertools import combinations


In [323]:
#CARGAR MATRIZ INCIDENCIA RANDOM, CONTROLO EL % DE CONEXIONES(1)
def matrizIncidencia(cantNodos:int, probabilidad:float):
    matriz=np.zeros((cantNodos,cantNodos),int)
    for fila in range(cantNodos):
        for columna in range(cantNodos):
            if fila==columna:
                matriz[fila][columna]=0
            elif random.random()<probabilidad:
                #CONTROLO CANT DE 1
                matriz[fila][columna]=1
    return matriz

In [324]:
#DE MATRIZ DE INCIDENCIA HAGO LISTA DE CONEXIONES
def listaConexiones(matriz:np.ndarray):
    lista=[]
    cantNodos=matriz.shape[0]
    for fila in range(cantNodos):
        columna=-1
        conexion=False
        for elemento in matriz[fila]:
            #ME EMPIEZO A MOVER X LAS COLUMNAS
            columna+=1
            if elemento==1:
                lista.append((fila,columna))
                conexion=True
        if conexion==False:
            #NODO SIN ARISTAS
            lista.append((fila,None))
    return lista

In [325]:
class TDAGrafo:
    def __init__(self,listaConexiones):
        self.grafo=dict()
        self.listaADiccionario(listaConexiones)
        self.cardinalesDicc=dict()
        self.cardinalidades()


    #DE LISTA DE CONEXIONES A DICCIONARIO
    def listaADiccionario(self,listaConexiones):
    
        for element1,element2 in listaConexiones:
            if element2 !=None:

                self.grafo.setdefault(element1,[]) #ESTRUCTURA
                if element2 not in self.grafo[element1]: #NO DUPLICADOS
                    self.grafo[element1].append(element2)

                self.grafo.setdefault(element2,[]) #ESTRUCTURA
                if element1 not in self.grafo[element2]: #NO DUPLICADOS
                    self.grafo[element2].append(element1)

            else:
                self.grafo.setdefault(element1,[])

    #ARMO DICCIONARIO CON CARDINALIDADES DE ARISTAS Y COMO VALOR: LOS NODOS CON ESA CARDINALIDAD
    def cardinalidades(self):
    
        for nodo,conexiones in self.grafo.items(): #ITENS TRAE CLAVE Y VALOR
            self.cardinalesDicc.setdefault(len(conexiones),[]).append(nodo) #ESTRUCTURA
        
       

    #HACE TODAS LAS COMBINACIONES DE LOS VERTICES DE UNA CARDINALIDAD Y DICE SI SON CLIQUE(Y LO DEVUELVE)
    def combinacionEsClique(self,vertices, cardiClique):
        combinaciones=list(combinations(vertices, cardiClique))
        print('COMBINACIONES:{}, {}'.format(combinaciones,cardiClique)) 
        for combinacion in combinaciones:
            if self.esClique(combinacion):
                return combinacion
        return False
        

    #DADA UNA COMBINACION SE FIJA SI ENTRE ELLOS SON VECINOS
    def esClique(self,combinacion):
        for nodo in combinacion:
            for elemento in combinacion:
                if nodo != elemento:
                    if nodo not in self.grafo[elemento]:
                        return False
        return True





In [326]:
#DADO UN VALOR, DEVUELVE UN ARRAY CON TODOS LOS NODOS DE ESA CARDINALIDAD PARA ARRIBA INCLUSIVE
def nodosSuperiores(grafo, valor):
    lista=list(grafo.cardinalesDicc.keys())
    lista.sort()
    print('DICCIONARIO CARDINALES LISTA:{} '.format(lista))
    arrayNodosSup=[]
    arrayCardinalesSup=[]
    for cardi in lista:
        if cardi>=valor:
            arrayCardinalesSup.append(cardi)
    
    print('ARRAY CARDINALES SUP:{} '.format(arrayCardinalesSup))
    
    for card in arrayCardinalesSup:
         for nodo in grafo.cardinalesDicc[card]:
                arrayNodosSup.append(nodo)
    print('Nodos Superiores para cardinalidad {}: {}'.format(valor,arrayNodosSup))
    return arrayNodosSup

In [327]:
def cliqueMaxHeuristic(grafo:TDAGrafo):
    cardiordenado=sorted(grafo.cardinalesDicc)
    gradoMax=cardiordenado[-1] #TRAE EL GRADO MAXIMO DE LOS NODOS DEL GRAFO
    limInf=0
    limSup=gradoMax
    print('Limite Superior:'+str(limSup))
    cliqueMax=None
    mitad=(limInf+limSup)//2

    while limInf<=limSup:
        nodosSuper=nodosSuperiores(grafo,mitad) # DE MITAD PARA ARRIBA INCLUSIVE
        cantNodosSup=len(nodosSuper)
        if cantNodosSup>mitad: #TENGO NODOS PARA ANALIZAR EN LA PARTE DERECHA
            cliqueAux=grafo.combinacionEsClique(nodosSuper, mitad+1) #BUSCO CLIQUE DE GRADO MITAD+1
            if cliqueAux: #SI HAY BUSCO MAS GRANDES

                cliqueMax=cliqueAux
                limInf=mitad+1
            else:
                limSup=mitad-1 #SI NO HAY BUSCO DE LA MITAD INFERIOR
        else: 
            limSup=mitad-1 #
        mitad=(limInf+limSup)//2 #ACTUALIZO LA MITAD SIEMPRE
    return cliqueMax   
        



In [328]:
cantNodos=5
probabilidad=0.5



m1 = np.array([
    [0,1,1,1,0,1,1,0],
    [1,0,1,1,1,1,0,0],
    [1,1,0,1,0,0,1,1],
    [1,1,1,0,1,0,0,1],
    [0,1,0,1,0,0,0,0],
    [1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0],
    [0,0,1,1,0,0,0,0]])

m=matrizIncidencia(cantNodos,probabilidad)

m2= np.array([
    [0,1,1,1,1],
    [1,0,1,1,1],
    [1,1,0,1,1],
    [1,1,1,0,1],
    [1,1,1,1,0]
])

m= np.array([
    [0,1,0,0,0],
    [1,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])

print(m)


l=listaConexiones(m)
#print(l)
print()

grafo=TDAGrafo(l)
print()
pretty_print = pprint.PrettyPrinter()
pretty_print.pprint(grafo.grafo)
print('diccionario Cardinalidades')
pretty_print = pprint.PrettyPrinter()
pretty_print.pprint(grafo.cardinalesDicc)


resultado=cliqueMaxHeuristic(grafo)

print()
print(resultado)

[[0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


{0: [1], 1: [0], 2: [], 3: [], 4: []}
diccionario Cardinalidades
{0: [2, 3, 4], 1: [0, 1]}
Limite Superior:1
DICCIONARIO CARDINALES LISTA:[0, 1] 
ARRAY CARDINALES SUP:[0, 1] 
Nodos Superiores para cardinalidad 0: [2, 3, 4, 0, 1]
COMBINACIONES:[(2,), (3,), (4,), (0,), (1,)], 1
DICCIONARIO CARDINALES LISTA:[0, 1] 
ARRAY CARDINALES SUP:[1] 
Nodos Superiores para cardinalidad 1: [0, 1]
COMBINACIONES:[(0, 1)], 2

(0, 1)


In [329]:
calvesordenadas = sorted({1:1,89:7,22:9,4:34,3:90})
print(calvesordenadas[-1])
calvesordenadas

89


[1, 3, 4, 22, 89]

In [330]:
valor=2
listita=[0,1,2,3,4,5]
print(listita[valor-1:])

[1, 2, 3, 4, 5]
